In [21]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
from pandas import to_datetime

In [22]:
#read data
train = pd.read_csv('/content/drive/MyDrive/personal project/train.csv', index_col=[0], parse_dates=[0])
test = pd.read_csv('/content/drive/MyDrive/personal project/test.csv', index_col=[0], parse_dates=[0])
submission = pd.read_csv('/content/drive/MyDrive/personal project/sampleSubmission.csv', index_col=[0], parse_dates=[0])

In [23]:
def createdd_train(df):
    """
    change the date from string into datetim
    """
    df['date'] = to_datetime(df['date'], format="%d/%m/%Y %H:%M")
    df['speed'] = df['speed']
    X = df[['date', 'speed']]
    return X


In [24]:
def createdd_test(df):
    """
    change the date from string into datetime
    """
    df['date'] = to_datetime(df['date'], format="%d/%m/%Y %H:%M")
    X = df[['date']]
    return X

In [25]:
train_dd = createdd_train(train)

In [26]:
test_dd = createdd_test(test)

In [27]:
#set 'date' as index
train_dd.index = train_dd['date']
test_dd.index = test_dd['date']

In [28]:
train_dd.head()

,date,speed
date,,
2017-01-01 00:00:00,2017-01-01 00:00:00,43.002930
2017-01-01 01:00:00,2017-01-01 01:00:00,46.118696
2017-01-01 02:00:00,2017-01-01 02:00:00,44.294158
2017-01-01 03:00:00,2017-01-01 03:00:00,41.067468
2017-01-01 04:00:00,2017-01-01 04:00:00,46.448653


In [29]:
test_dd.head()

,date
date,
2018-01-01 02:00:00,2018-01-01 02:00:00
2018-01-01 05:00:00,2018-01-01 05:00:00
2018-01-01 07:00:00,2018-01-01 07:00:00
2018-01-01 08:00:00,2018-01-01 08:00:00
2018-01-01 10:00:00,2018-01-01 10:00:00


In [30]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X


In [31]:
X_train, y_train = create_features(train_dd, label='speed')
X_test = create_features(test_dd)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


In [32]:
train_dd.head()

,date,speed,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,,
2017-01-01 00:00:00,2017-01-01 00:00:00,43.002930,0,6,1,1,2017,1,1,52
2017-01-01 01:00:00,2017-01-01 01:00:00,46.118696,1,6,1,1,2017,1,1,52
2017-01-01 02:00:00,2017-01-01 02:00:00,44.294158,2,6,1,1,2017,1,1,52
2017-01-01 03:00:00,2017-01-01 03:00:00,41.067468,3,6,1,1,2017,1,1,52
2017-01-01 04:00:00,2017-01-01 04:00:00,46.448653,4,6,1,1,2017,1,1,52


In [33]:
X_test.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,
2018-01-01 02:00:00,2,0,1,1,2018,1,1,1
2018-01-01 05:00:00,5,0,1,1,2018,1,1,1
2018-01-01 07:00:00,7,0,1,1,2018,1,1,1
2018-01-01 08:00:00,8,0,1,1,2018,1,1,1
2018-01-01 10:00:00,10,0,1,1,2018,1,1,1


In [34]:
X_train.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,
2017-01-01 00:00:00,0,6,1,1,2017,1,1,52
2017-01-01 01:00:00,1,6,1,1,2017,1,1,52
2017-01-01 02:00:00,2,6,1,1,2017,1,1,52
2017-01-01 03:00:00,3,6,1,1,2017,1,1,52
2017-01-01 04:00:00,4,6,1,1,2017,1,1,52


In [35]:
#train the model
reg = xgb.XGBRegressor(n_estimators=10000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train)],
        early_stopping_rounds=50,
       verbose=False)

[11:22:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=10000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [19]:
#prediction
submission['speed'] = reg.predict(X_test)

In [20]:
#output the result
submission.to_csv('/content/drive/MyDrive/personal project/resulting test.csv')